In [2]:
# sourcery skip: for-index-underscore
import contextlib
import pandas as pd
import requests
from datetime import *
from bs4 import BeautifulSoup

In [3]:
url = "https://www.tmz.com"
req = requests.get(url)
doc = BeautifulSoup(req.text)

In [4]:
# init var headline
tmz = doc.select("header a h2")

In [5]:
# init var timestamp
tmz_timestamps = doc.select(".article")

In [95]:
tmz_tags = doc.select(".article .tag-cloud ul a")

In [103]:
tmz_tags = doc.select(".tag-cloud__list-group")
tmz_tags[0]

<ul class="tag-cloud__list-group">
<a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":1}' href="https://www.tmz.com/people/shanna-moakler/">
                    Shanna Moakler
                  </a>
<a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":1}' href="https://www.tmz.com/categories/reality-tv/">
                    Reality TV
                  </a>
<a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":1}' href="https://www.tmz.com/categories/relationships/">
                    Relationships
                  </a>
<a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":1}' href="https://www.tmz.com/categories/couples/">
                    Couples
                  </a>
<a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name

In [120]:
tmz_tags[25]

<a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":5}' href="https://www.tmz.com/people/dwight-howard/">
                    Dwight Howard
                  </a>

In [92]:
count = 0
tags = tmz_tags[count].text
tmz_tags = doc.select(".article .tag-cloud ul a")
tag_list = []
tag_counter = 0
for tags in enumerate(tmz_tags):
    tag = tmz_tags[tag_counter].get_text(strip=True)
    tag_list.append(tag)
    tag_counter += 1

KeyError: 0

In [136]:
tmz_tags = doc.select(".article .tag-cloud__list-group .tag-cloud__list-group-item")
len(tmz_tags)


99

In [ ]:
//*[@id="blogroll-b1c1be28-2d33-4b07-91de-b058bf157b91"]/section[2]/ul/a[1]

In [146]:
tmz_tags = doc.select(".blogroll section ul a pos.1")
tmz_tags

SelectorSyntaxError: Malformed class selector at position 26
  line 1:
.blogroll section ul a pos.1
                          ^

In [142]:
tmz_tags = doc.select(".blogroll section.tag-cloud > ul > a:nth-child(1)")
tmz_tags


[<a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":1}' href="https://www.tmz.com/people/shanna-moakler/">
                     Shanna Moakler
                   </a>,
 <a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":2}' href="https://www.tmz.com/categories/animals/">
                     Animals
                   </a>,
 <a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":3}' href="https://www.tmz.com/people/jojo-siwa/">
                     Jojo Siwa
                   </a>,
 <a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":4}' href="https://www.tmz.com/people/theresa-caputo/">
                     Theresa Caputo
                   </a>,
 <a class="tag-cloud__list-group-item js-track-link" data-context='{"section":"main","name":"tag-cloud","pos":5}' hr

In [ ]:
#blogroll-b1c1be28-2d33-4b07-91de-b058bf157b91 > section.tag-cloud > ul > a:nth-child(1)

In [107]:
# main loop
stories = pd.DataFrame()
paragraphs = []
count = 0

for trash in enumerate(tmz):
    
    # grab headline text
    headline = tmz[count].text
    headline = headline.replace("\n", " ")
    
    # grab tags
    tmz_tags = doc.select(".tag-cloud__list-group")
    tags = tmz_tags[count].text
    tag_list = []
    tag_counter = 0
    for tags in enumerate(tmz_tags):
        tag = tmz_tags[tag_counter].get_text(strip=True)
        tag_list.append(tag)
        tag_counter += 1

    # grab, clean timestamp
    timestamp = tmz_timestamps[count].text.split("PT")[-20:]
    timestamp = timestamp[0][-20:]
    timestamp = timestamp.strip()

    # set timestamp, date, time, headlie
    story = {
        "timestamp": timestamp,
        "date": timestamp[:9],
        "time": timestamp[-8:].strip(),
        "headline": headline,
        "tags": tag_list
    }

    # convert to df
    story = pd.DataFrame([story])

    # convert story 12AP to 24
    try:
        story["time"] = pd.to_datetime(
            story["time"], format="%I:%M %p"
        ).dt.strftime("%H:%M")
    except:
        story["time"] = story["time"]
        
    story.convert_dtypes()
    
    story['year'] = pd.DatetimeIndex(story['date']).year
    story['month'] = pd.DatetimeIndex(story['date']).month
    story['day'] = pd.DatetimeIndex(story['date']).day
    story['hour'] = pd.DatetimeIndex(story['time']).hour
    story['minute'] = pd.DatetimeIndex(story['time']).minute
    # parse date into columns
    # try:
    #     story["year"] = story['date'].astype(str)[-4:]
    # except:
    #     story["year"] = story["year"]

    # append story to raw stories df to be merged with historical df
    stories = pd.concat([stories, story], ignore_index=True)

    count += 1

stories = stories[["headline", "tags", "timestamp", "year", "month", "day", "hour", "minute"]]
stories

,headline,tags,timestamp,year,month,day,hour,minute
0,Shanna Moakler Cops Respond to 'Domestic Dist...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 6:31 PM,2022,7,28,18,31
1,Monkey Attacks Terrorizing Japanese City ... ...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 6:20 PM,2022,7,28,18,20
2,Candace Cameron Bure Vs. JoJo Siwa Daughter R...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 5:53 PM,2022,7,28,17,53
3,'Long Island Medium' Theresa Defends Buying 5...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 4:19 PM,2022,7,28,16,19
4,Dwight Howard Crashes WWE Tryout ... Cuts Epi...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 3:36 PM,2022,7,28,15,36
5,Beyoncé Catches R&Beef From Kelis!!! Claims S...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 1:41 PM,2022,7,28,13,41
6,Report: Zion Williamson Weight Clause In New ...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 2:58 PM,2022,7,28,14,58
7,"Joey Badass Fans Gotta Chill, Chris Brown Has...",[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 3:33 PM,2022,7,28,15,33
8,Lamar Odom My Social Media's Been Highjacked ...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 12:00 PM,2022,7,28,12,0
9,Jon Stewart Obliterates GOP for Blocking Vet ...,[Shanna MoaklerReality TVRelationshipsCouplesE...,7/28/2022 10:26 AM,2022,7,28,10,26


In [18]:
existing = pd.read_csv("../headlines.csv")
existing = existing.dropna(axis=1).set_index("Unnamed: 0")
existing.convert_dtypes()

existing["year"] = pd.DatetimeIndex(existing["timestamp"]).year
existing["month"] = pd.DatetimeIndex(existing["timestamp"]).month
existing["day"] = pd.DatetimeIndex(existing["timestamp"]).day
existing["hour"] = pd.DatetimeIndex(existing["timestamp"]).hour
existing["minute"] = pd.DatetimeIndex(existing["timestamp"]).minute

existing

,headline,timestamp,year,month,day,hour,minute
Unnamed: 0,,,,,,,
158,Jamie Foxx Wild Sing-along on Italian Vacay ....,7/27/2022 8:02 AM,2022,7,27,8,2
157,Jerry Jones Apologizes For Derogatory Term .....,7/26/2022 4:23 PM,2022,7,26,16,23
155,'Top Chef' Star Howard Kleinberg Dead at 46,7/27/2022 7:39 AM,2022,7,27,7,39
156,Norman Lear Singin' 'That's Amore' At 100 ......,7/27/2022 8:14 AM,2022,7,27,8,14
0,Mike Tyson Using Cane To Get Around ... But I...,7/27/2022 3:56 PM,2022,7,27,15,56
...,...,...,...,...,...,...,...
150,Ukrainian Prez Volodymyr Zelensky Dragged Ove...,7/27/2022 10:15 AM,2022,7,27,10,15
151,GEORGE FLOYD'S DEATH Former Cops Get Prison T...,7/27/2022 10:14 AM,2022,7,27,10,14
152,Shawn Mendes Cancels Tour ... For Mental Heal...,7/27/2022 7:17 AM,2022,7,27,7,17


In [32]:
try:
    existing = pd.read_csv("headlines.csv")
    existing.convert_dtypes()
    existing["year"] = pd.DatetimeIndex(existing["timestamp"]).year
    existing["month"] = pd.DatetimeIndex(existing["timestamp"]).month
    existing["day"] = pd.DatetimeIndex(existing["timestamp"]).day
    existing["hour"] = pd.DatetimeIndex(existing["timestamp"]).hour
    existing["minute"] = pd.DatetimeIndex(existing["timestamp"]).minute
    existing = existing[
        ["headline", "timestamp", "date", "time", "year", "month", "day", "hour", "minute"]
    ]
except:
    pass
    
export = pd.concat([existing, stories], ignore_index=True)
export = export[
    ["headline", "timestamp", "year", "month", "day", "hour", "minute"]
]

export = export.drop_duplicates(subset=['timestamp'], keep="first")
export = export.sort_values(by=["year", "month", "day", "hour", "minute"], ascending=[False, False, False, False, False], ignore_index=True)
export

,headline,timestamp,year,month,day,hour,minute
0,Mike Tyson Using Cane To Get Around ... But I...,7/27/2022 3:56 PM,2022,7,27,15,56
1,"Britney Spears Will Not Be Deposed, Judge Rules",7/27/2022 3:26 PM,2022,7,27,15,26
2,Jeff Bezos Parents Buy $34M Mega Mansion in FL,7/27/2022 3:01 PM,2022,7,27,15,1
3,Joey Chestnut Downs 44 Cane's Chicken Fingers...,7/27/2022 2:06 PM,2022,7,27,14,6
4,Brittney Griner I Flew To Russia Despite Trav...,7/27/2022 1:06 PM,2022,7,27,13,6
5,Tony Dow 'Leave It To Beaver' Star Dead At 77,7/27/2022 12:27 PM,2022,7,27,12,27
6,Beyoncé ‘Renaissance’ Album Leak Beyhive Says...,7/27/2022 12:02 PM,2022,7,27,12,2
7,Ukrainian Prez Volodymyr Zelensky Dragged Ove...,7/27/2022 10:15 AM,2022,7,27,10,15
8,GEORGE FLOYD'S DEATH Former Cops Get Prison T...,7/27/2022 10:14 AM,2022,7,27,10,14
9,Kanye West Gifts A$AP Bari New Maybach… Days ...,7/27/2022 8:18 AM,2022,7,27,8,18


In [33]:
export.to_csv("headlines.csv")